# Homework 1 (Total Points: 260) <a class="anchor" id="top"></a>


**Submission instructions**:
- Only the code `TODO: Implement this!` denotes that these sections are graded.
- For Part 1: You can use the `nltk`, `numpy` and `matplotlib` libraries here. Other libraries, e.g., `gensim` or `scikit-learn`, may not be used. For Part 2: `gensim` is allowed in addition to the imported libraries in the next code cell
- The notebook you submit has to have the student ids, separated by underscores (E.g., `12341234_12341234_12341234.ipynb`). 
- This will be parsed by a regexp, so please double check your filename.
- Only one member of each group has to submit the file to canvas.
- Make sure to check that your notebook runs before submission. A quick way to do this is to restart the kernel and run all the cells.  
- Please do not delete/add new cells. Removing cells can lead to grade deduction.
- Note, that you are not allowed to use Google Colab.


**Learning Goals**:
- [Part 1, Term-based matching](#part1) (175 points):
    - Learn how to load a dataset and process it.
    - Learn how to implement several standard IR methods (TF-IDF, BM25, QL) and understand their weaknesses & strengths.
    - Learn how to evaluate IR methods.
- [Part 2, Semantic-based matching](#part2) (85 points):
    - Learn how to implement vector-space retrieval methods (LSI, LDA).
    - Learn how to use LSI and LDA for re-ranking

    
**Resources**: 
- **Part 1**: Sections 2.3, 4.1, 4.2, 4.3, 5.3, 5.6, 5.7, 6.2, 7, 8 of [Search Engines: Information Retrieval in Practice](https://ciir.cs.umass.edu/downloads/SEIRiP.pdf)
- **Part 2**: [LSI - Chapter 18](https://nlp.stanford.edu/IR-book/pdf/18lsi.pdf) from [Introduction to Information Retrieval](https://nlp.stanford.edu/IR-book/) book and the [original LDA paper](https://jmlr.org/papers/volume3/blei03a/blei03a.pdf)

In [1]:
# imports 
# TODO: Ensure that no additional library is imported in the notebook. 
# TODO: Only the standard library and the following libraries are allowed:
# TODO: You can also use unlisted classes from these libraries or standard libraries (such as defaultdict, Counter, ...).

import os
import zipfile
from functools import partial

import nltk
import requests
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

from ipywidgets import widgets
from IPython.display import display, HTML
#from IPython.html import widgets
from collections import namedtuple

%matplotlib inline


# Part 1: Term-based Matching (175 points) <a class="anchor" id="part1"></a>

[Back to top](#top)

In the first part, we will learn the basics of IR from loading and preprocessing the material, to implementing some well known search algorithms, to evaluating the ranking performance of the implemented algorithms. We will be using the CACM dataset throughout the assignment. The CACM dataset is a collection of titles and abstracts from the journal CACM (Communication of the ACM).

Table of contents:
- [Section 1: Text Processing](#text_processing) (15 points)
- [Section 2: Indexing](#indexing) (10 points)
- [Section 3: Ranking](#ranking) (80 points)
- [Section 4: Evaluation](#evaluation) (40 points)
- [Section 5: Analysis](#analysis) (30 points)


---
## Section 1: Text Processing (15 points)<a class="anchor" id="text_processing"></a>

[Back to Part 1](#part1)

In this section, we will load the dataset and learn how to clean up the data to make it usable for an IR system. 
The points of this section are earned for the following implementations:
- `read_cacm_docs` (4 points): Reads in the CACM documents.
- `read_queries` (3 points): Reads in the CACM queries.
- `load_stopwords` (1 point): Loads the stopwords.
- `tokenize` (4 points): Tokenizes the input text.
- `stem_token` (3 points): Stems the given token. 

We are using the [CACM dataset](http://ir.dcs.gla.ac.uk/resources/test_collections/cacm/), which is a small, classic IR dataset, composed of a collection of titles and abstracts from the journal CACM. It comes with relevance judgements for queries, so we can evaluate our IR system. 


---
### 1.1 Read the CACM documents (4 points)


The following cell downloads the dataset and unzips it to a local directory.

In [2]:
def download_dataset():
    folder_path = os.environ.get("IR1_DATA_PATH")
    if not folder_path:
        folder_path = "./datasets/"
    os.makedirs(folder_path, exist_ok=True)
    
    file_location = os.path.join(folder_path, "cacm.zip")
    
    # download file if it doesn't exist
    if not os.path.exists(file_location):
        
        url = "https://surfdrive.surf.nl/files/index.php/s/M0FGJpX2p8wDwxR/download"

        with open(file_location, "wb") as handle:
            print(f"Downloading file from {url} to {file_location}")
            response = requests.get(url, stream=True)
            for data in tqdm(response.iter_content()):
                handle.write(data)
            print("Finished downloading file")
    
    if not os.path.exists(os.path.join(folder_path, "train.txt")):
        
        # unzip file
        with zipfile.ZipFile(file_location, 'r') as zip_ref:
            zip_ref.extractall(folder_path)
        
download_dataset()

---

You can see a brief description of each file in the dataset by looking at the README file:

In [3]:
##### Read the README file 
!cat ./datasets/README
#####

Files in this directory with sizes:
          0 Jun 19 21:01 README

    2187734 Jun 19 20:55 cacm.all              text of documents
        626 Jun 19 20:58 cite.info             key to citation info
                                                (the X sections in cacm.all)
       2668 Jun 19 20:55 common_words           stop words used by smart
       2194 Jun 19 20:55 make_coll*             shell script to make collection
       1557 Jun 19 20:55 make_coll_term*        ditto (both useless without
                                                smart system)
       9948 Jun 19 20:55 qrels.text             relation giving
                                                    qid did 0 0
                                                to indicate dument did is
                                                relevant to query qid
      13689 Jun 19 20:55 query.text             Original text of the query


---
We are interested in 4 files:
- `cacm.all` : Contains the text for all documents. Note that some documents do not have abstracts available. 
- `query.text` : The text of all queries
- `qrels.text` : The relevance judgements
- `common_words` : A list of common words. This may be used as a collection of stopwords

In [4]:
##### The first 45 lines of the CACM dataset forms the first record
# We are interested only in 3 fields. 
# 1. the '.I' field, which is the document id
# 2. the '.T' field (the title) and
# 3. the '.W' field (the abstract, which may be absent)
!head -45 ./datasets/cacm.all
#####

.I 1
.T
Preliminary Report-International Algebraic Language
.B
CACM December, 1958
.A
Perlis, A. J.
Samelson,K.
.N
CA581203 JB March 22, 1978  8:28 PM
.X
100	5	1
123	5	1
164	5	1
1	5	1
1	5	1
1	5	1
205	5	1
210	5	1
214	5	1
1982	5	1
398	5	1
642	5	1
669	5	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
1	6	1
165	6	1
196	6	1
196	6	1
1273	6	1
1883	6	1
324	6	1
43	6	1
53	6	1
91	6	1
410	6	1
3184	6	1


---

**Implementation (4 points):**
Write a function to read the `cacm.all` file. Note that each document has a variable number of lines. The `.I` field denotes a new document:

In [5]:
# TODO: Implement this! (4 points)
# Complexity: O(n)

def read_cacm_docs(root_folder = "./datasets/"):
    """
        Reads in the CACM documents. The dataset is assumed to be in the folder "./datasets/" by default
        Returns: A list of 2-tuples: (doc_id, document), where 'document' is a single string created by 
            appending the title and abstract (separated by a "\n"). 
            In case the record doesn't have an abstract, the document is composed only by the title
    """
    
    # Path to cacm.all
    path = root_folder + "cacm.all"
    
    # Specify lists for output
    doc_id = []
    doc_title = []
    
    # Open and split document into tokens
    with open(path, 'r') as file:
        doclist = file.read().split("\n")

    # Set counter for index tracking
    counter = 0
    i = 0
    
    # Loop through doc to get I., T. and W.
    while i < len(doclist) - 1:
        counter += 1
        
        # Append document id and title
        doc_id.append(int(doclist[i][3:]))
        doc_title.append(doclist[i+2])
        
        i += 3
            
        while doclist[i] not in [".B", ".A", ".X"]:
            if doclist[i] == '.W':
                i += 1
                doc_title[counter-1] += " \n"
            else:
                doc_title[counter-1] += " " + doclist[i]
                i += 1
        
        while doclist[i][:2] != ".I" and i < len(doclist)-1:
            i += 1
    
    return list(zip(doc_id, doc_title))

In [6]:
##### Function check
docs = read_cacm_docs()

assert isinstance(docs, list)
assert len(docs) == 3204, "There should be exactly 3024 documents"
##### 

---
### 1.2 Read the CACM queries (3 points)

Next, let us read the queries. They are formatted similarly:

In [7]:
##### The first 15 lines of 'query.text' has 2 queries
# We are interested only in 2 fields. 
# 1. the '.I' - the query id
# 2. the '.W' - the query
!head -15 ./datasets/query.text
#####

.I 1
.W
 What articles exist which deal with TSS (Time Sharing System), an
operating system for IBM computers?
.N
 1. Richard Alexander, Comp Serv, Langmuir Lab (TSS)
 
.I 2
.W
 I am interested in articles written either by Prieve or Udo Pooch
.A
Prieve, B.
Pooch, U.
.N
 2. Richard Alexander, Comp Serv, Langmuir Lab (author = Pooch or Prieve)


---

**Implementation (3 points):**
Write a function to read the `query.text` file:

In [8]:
# TODO: Implement this! (3 points)
def read_queries(root_folder = "./datasets/"):
    """
        Reads in the CACM queries. The dataset is assumed to be in the folder "./datasets/" by default
        Returns: A list of 2-tuples: (query_id, query)
    """
    
    # Path to query.text
    path = root_folder + "query.text"
    
    # Open and split document into tokens
    with open(path, 'r') as file:
        queries = file.read().split("\n")
    
    # Define lists
    query_id = []
    query = []
    
    # Set counter for tracking index tracking
    counter = 0
    
    # Loop through doc to get I. and W.
    for i in range(len(queries)):
        if queries[i][0:2] == ".I":
            counter += 1
            query_id.append(int(queries[i][3:]))
        if  queries[i] == '.W':
            query.append(" " + queries[i+1][1:])
            j = i+2
            while queries[j] not in [".N", ".A"]:
                query[counter-1] = query[counter-1] + " " + queries[j]
                j += 1
               
    
    return list(zip(query_id, query))

In [9]:
##### Function check
queries = read_queries()

assert isinstance(queries, list)
assert len(queries) == 64 and all([q[1] is not None for q in queries]), "There should be exactly 64 queries"
##### 

---
### 1.3 Read the stop words (1 point)

We use the common words stored in `common_words`:

In [10]:
##### Read the stop words file 
!head ./datasets/common_words
##### Read the README file 

a
about
above
accordingly
across
after
afterwards
again
against
all


---
**Implementation (1 point):**
Write a function to read the `common_words` file (For better coverage, try to keep them in lowercase):

In [11]:
# TODO: Implement this! (1 point)
def load_stopwords(root_folder = "./datasets/"):
    """
        Loads the stopwords. The dataset is assumed to be in the folder "./datasets/" by default
        Output: A set of stopwords
    """
    
    
    path = root_folder + "common_words"
    
    # Open and split document into tokens
    with open(path, 'r') as file:
        stopwords = file.read().lower().split('\n')[:-1]
    
    
    return set(stopwords)

In [12]:
##### Function check
stopwords = load_stopwords()

assert isinstance(stopwords, set)
assert len(stopwords) == 428, "There should be exactly 428 stop words"
##### 

---
### 1.4 Tokenization (4 points)

We can now write some basic text processing functions. 
A first step is to tokenize the text. 

**Note**: Use the  `WordPunctTokenizer` available in the `nltk` library:

In [13]:
# TODO: Implement this! (4 points)
def tokenize(text):
    """
        Tokenizes the input text. Use the WordPunctTokenizer
        Input: text - a string
        Output: a list of tokens
    """
    
    return nltk.WordPunctTokenizer().tokenize(text) 

In [14]:
##### Function check
text = "the quick brown fox jumps over the lazy dog"
tokens = tokenize(text)

assert isinstance(tokens, list)
assert len(tokens) == 9

print(tokens)
# output: ['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']
#####

['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']


---
### 1.5 Stemming (3 points)

Write a function to stem tokens. 
Again, you can use the nltk library for this:

In [15]:
# # importing modules 
# from nltk.stem import PorterStemmer 
# from nltk.tokenize import word_tokenize 
   
# ps = PorterStemmer() 
   
# sentence = "Programers program with programing languages"
# words = word_tokenize(sentence) 
   
# for w in words: 
#     print(w, " : ", ps.stem(w)) 

# TODO: Implement this! (3 points)
def stem_token(token):
    """
        Stems the given token using the PorterStemmer from the nltk library
        Input: a single token
        Output: the stem of the token
    """
    
    return nltk.PorterStemmer().stem(token)

In [16]:
##### Function check

assert stem_token('owned') == 'own'
assert stem_token('itemization') == 'item'
#####

---
### 1.6 Summary

The following function puts it all together. Given a string, it tokenizes it and processes it according to the flags that you set.

In [17]:
#### Putting it all together
def process_text(text, stem=False, remove_stopwords=False, lowercase_text=False):
    
    tokens = []
    for token in tokenize(text):
        if remove_stopwords and token.lower() in stopwords:
            continue
        if stem:
            token = stem_token(token)
        if lowercase_text:
            token = token.lower()
        tokens.append(token)

    return tokens
#### 

---

Let's create two sets of preprocessed documents.
We can process the documents and queries according to these two configurations:

In [18]:
# In this configuration:
# Don't preprocess the text, except to tokenize 
config_1 = {
  "stem": False,
  "remove_stopwords" : False,
  "lowercase_text": True
} 


# In this configuration:
# Preprocess the text, stem and remove stopwords
config_2 = {
  "stem": True,
  "remove_stopwords" : True,
  "lowercase_text": True, 
} 

####
doc_repr_1 = []
doc_repr_2 = []
for (doc_id, document) in docs:
    doc_repr_1.append((doc_id, process_text(document, **config_1)))
    doc_repr_2.append((doc_id, process_text(document, **config_2)))

####

--- 

## Section 2: Indexing (10 points)<a class="anchor" id="indexing"></a>

[Back to Part 1](#part1)



A retrieval function usually takes in a query document pair, and scores a query against a document.  Our document set is quite small - just a few thousand documents. However, consider a web-scale dataset with a few million documents. In such a scenario, it would become infeasible to score every query and document pair. In such a case, we can build an inverted index. From Wikipedia:

> ... , an inverted index (also referred to as a postings file or inverted file) is a database index storing a mapping from content, such as words or numbers, to its locations in a table, .... The purpose of an inverted index is to allow fast full-text searches, at a cost of increased processing when a document is added to the database. ...


Consider a simple inverted index, which maps from word to document. This can improve the performance of a retrieval system significantly. In this assignment, we consider a *simple* inverted index, which maps a word to a set of documents. In practice, however, more complex indices might be used.  


### 2.1 Term Frequency-index (10 points)
In this assignment, we will be using an index created in memory since our dataset is tiny. To get started, build a simple index that maps each `token` to a list of `(doc_id, count)` where `count` is the count of the `token` in `doc_id`.
For consistency, build this index using a python dictionary.
    
Now, implement a function to build an index:

In [19]:
def build_tf_index(documents):
    """
        Build an inverted index (with counts). The output is a dictionary which takes in a token
        and returns a list of (doc_id, count) where 'count' is the count of the 'token' in 'doc_id'
        Input: a list of documents - (doc_id, tokens)
        Output: An inverted index. [token] -> [(doc_id, token_count)]
    """
    
    inv_index = {}

    for document in documents:
        
        # Create dict to store token info for this document
        hashtable = {}
        doc_id = int(document[0])
        
        # Go through all tokens and count them
        for token in document[1]:
            
            # Initialize or increase count
            if token not in hashtable:
                hashtable[token] = 1
            else:
                hashtable[token] += 1
                
        # Go through tokens and append their count to the index
        for token, count in hashtable.items():
            if token not in inv_index:
                inv_index[token] = []
            inv_index[token].append((doc_id, count))
            
    return inv_index

---
Now we can build indexed documents and preprocess the queries based on the two configurations:

In [20]:
#### Indexed documents based on the two configs

# Create the 2 indices
tf_index_1 = build_tf_index(doc_repr_1)
tf_index_2 = build_tf_index(doc_repr_2)

# This function returns the tf_index of the corresponding config
def get_index(index_set):
    assert index_set in {1, 2}
    return {
        1: tf_index_1,
        2: tf_index_2
    }[index_set]

####
#### Preprocessed query based on the two configs

# This function preprocesses the text given the index set, according to the specified config
def preprocess_query(text, index_set):
    assert index_set in {1, 2}
    if index_set == 1:
        return process_text(text, **config_1)
    elif index_set == 2:
        return process_text(text, **config_2)

#### 

In [21]:
##### Function check

print(tf_index_1['computer'][:10])
#### 

[(4, 1), (7, 1), (10, 1), (13, 1), (19, 1), (22, 1), (23, 1), (37, 1), (40, 3), (41, 1)]


In [22]:
##### Function check

print(tf_index_2['computer'][:10])
#### 

[(670, 1), (675, 1), (1621, 3), (1681, 1), (2145, 1), (2339, 1), (2572, 1), (2583, 1), (2739, 1), (3012, 1)]



---
## Section 3: Ranking  (80 points) <a class="anchor" id="ranking"></a>

[Back to Part 1](#part1)

Now that we have cleaned and processed our dataset, we can start building simple IR systems. 

For now, we consider *simple* IR systems, which involve computing scores from the tokens present in the document/query. More advanced methods are covered in later assignments.

We will implement the following methods in this section:
- [Section 3.1: Bag of Words](#bow) (10 points)
- [Section 3.2: TF-IDF](#tfidf) (15 points)
- [Section 3.3: Query Likelihood Model](#qlm) (35 points)
- [Section 3.4: BM25](#bm25) (20 points)

**Scoring policy:**
Your implementations in this section are scored based on the expected performance of your ranking functions.
You will get a full mark if your implementation meets the expected performance (measured by some evaluation metric).
Otherwise, you may get partial credit.
For example, if your *Bag of words* ranking function has 60% of expected performance, you will get 4 out of 10.

--- 

### Section 3.1: Bag of Words (10 points)<a class="anchor" id="bow"></a>

Probably the simplest IR model is the Bag of Words (BOW) model.
Implement a function that scores and ranks all the documents against a query using this model.   

Note that you can use either the count of the token or 'binarize' it i.e set the value equal to 1 if the token appears.   


In [23]:
# TODO: Implement this! (10 points)
def bow_search(query, index_set):
    """
        Perform a search over all documents with the given query. 
        Note: You have to use the `get_index` function created in the previous cells
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    
    index = get_index(index_set)
    processed_query = preprocess_query(query, index_set)
    
    scores = {}
    for q in processed_query:
        for doc, count in index[q]:
            if doc not in scores:
                scores[doc] = 0
            scores[doc] += count
            
    return [(d, float(s)) for d, s in sorted(scores.items(), key=lambda t: t[1], reverse=True)]

In [24]:
#### Function check

docs_by_id = dict(docs)
def print_results(docs, len_limit=50):    
    for i, (doc_id, score) in enumerate(docs):
        doc_content = docs_by_id[doc_id].strip().replace("\n", "\\n")[:len_limit] + "..."
        print(f"Rank {i}({score:.2}): {doc_content}")

test_bow = bow_search("report", index_set=1)[:5]
print(f"BOW Results:")
print_results(test_bow)
#### 

BOW Results:
Rank 0(6.0): An Information Algebra - Phase I Report-Language S...
Rank 1(6.0): Rejuvenating Experimental Computer Science \n This...
Rank 2(3.0): ALGOL 60 Confidential \n The ALGOL 60 Report,* whe...
Rank 3(2.0): Automatic Abstracting and Indexing Survey and Reco...
Rank 4(2.0): A String Language for Symbol Manipulation Based on...


In [25]:
#### Please do not change this. This cell is used for grading.

In [26]:
#### Please do not change this. This cell is used for grading.

In [27]:
#### Please do not change this. This cell is used for grading.


---

### Section 3.2: TF-IDF (15 points) <a class="anchor" id="tfidf"></a>

Before we implement the tf-idf scoring functions, let's first write a function to compute the document frequencies of all words.  

#### 3.2.1 Document frequency (5 points)
Compute the document frequencies of all tokens in the collection.  

In [28]:
# TODO: Implement this! (5 points)

def compute_df(documents):
    
    """
        Compute the document frequency of all terms in the vocabulary
        Input: A list of documents
        Output: A dictionary with {token: document frequency)
    """
    
    df = dict()
    
    for document in documents:
        for token in set(document):
            if token in df:
                df[token] += 1
            else:
                df[token] = 1
    
    return df

In [29]:
#### Compute df based on the two configs

# get the document frequencies of each document
df_1 = compute_df([d[1] for d in doc_repr_1])
df_2 = compute_df([d[1] for d in doc_repr_2])

def get_df(index_set):
    assert index_set in {1, 2}
    return {
        1: df_1,
        2: df_2
    }[index_set]
####

In [30]:
#### Function check

print(df_1['computer'])
print(df_2['computer'])
####

597
11


---
#### 3.2.2 TF-IDF search (10 points)
Next, implement a function that computes a tf-idf score given a query.      

In [31]:
# TODO: Implement this! (10 points)

def tfidf_search(query, index_set):
    
    """
        Perform a search over all documents with the given query using tf-idf. 
        Note #1: You have to use the `get_index` (and the `get_df`) function created in the previous cells
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    
    index = get_index(index_set)
    df = get_df(index_set)
    processed_query = preprocess_query(query, index_set)
    
    # Make a set of all involved documents
    documents = set([z[0] for z in [y for x in index.values() for y in x]])
    
    # Make dict of all involved documents, assign score to 0
    tf = dict({key: 0 for key in documents})
    
    # Create new dict with Inverse document frequency
    idf = {n: np.log(len(documents)/df[n]) for n in df.keys()}
    
    # Multiply tf * idf and sum to calculate score
    for token in processed_query:
        if token not in index.keys():
            continue
        for doc in index[token]:
            tf[doc[0]] += doc[1]*idf[token]
    
    # Sort and return list
    return sorted(tf.items(), key=lambda x:x[1], reverse=True)

In [32]:
#### Function check
test_tfidf = tfidf_search("report", index_set=1)[:5]
print(f"TFIDF Results:")
print_results(test_tfidf)
####

TFIDF Results:
Rank 0(2.3e+01): An Information Algebra - Phase I Report-Language S...
Rank 1(2.3e+01): Rejuvenating Experimental Computer Science \n This...
Rank 2(1.2e+01): ALGOL 60 Confidential \n The ALGOL 60 Report,* whe...
Rank 3(7.8): Automatic Abstracting and Indexing Survey and Reco...
Rank 4(7.8): A String Language for Symbol Manipulation Based on...


In [33]:
#### Please do not change this. This cell is used for grading.

In [34]:
#### Please do not change this. This cell is used for grading.

In [35]:
#### Please do not change this. This cell is used for grading.

--- 

### Section 3.3: Query Likelihood Model (35 points) <a class="anchor" id="qlm"></a>

In this section, you will implement a simple query likelihood model. 


#### 3.3.1 Naive QL (15 points)

First, let us implement a naive version of a QL model, assuming a multinomial unigram language model (with a uniform prior over the documents). 



In [36]:
#### Document length for normalization

def doc_lengths(documents):
    doc_lengths = {doc_id:len(doc) for (doc_id, doc) in documents}
    return doc_lengths

doc_lengths_1 = doc_lengths(doc_repr_1)
doc_lengths_2 = doc_lengths(doc_repr_2)

def get_doc_lengths(index_set):
    assert index_set in {1, 2}
    return {
        1: doc_lengths_1,
        2: doc_lengths_2
    }[index_set]
####

In [37]:
from collections import defaultdict

# TODO: Implement this! (15 points)

def naive_ql_search(query, index_set):
    
    """
        Perform a search over all documents with the given query using a naive QL model. 
        Note #1: You have to use the `get_index` (and get_doc_lengths) function created in the previous cells
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    
    index = get_index(index_set)
    doc_lengths = get_doc_lengths(index_set)
    processed_query = preprocess_query(query, index_set)
    
    # Make a set of all involved documents
    documents = set([z[0] for z in [y for x in index.values() for y in x]])
    
    # Specifiy defaultdict
    scores = defaultdict(list)
    
    # For each term in query
    for term in processed_query:
        
        # If term does not exist in vocab assign 0
        if term not in index:
            for document in documents:
                scores[document].append(0)
            continue
        
        # Collect al doc_ids from documents in which term occurs
        in_docs = [x[0] for x in index[term]]
        
        # For every document
        for document in documents:
            
            # If term occurs in this document append probability
            if document in in_docs:            
                scores[document].append((index[term][in_docs.index(document)][1]/doc_lengths[document]))
                
            # If term does not occur, append probability of 0
            else:
                scores[document].append(0)
    
    # Take product of all elements
    scores.update({n: np.prod([float(y) for y in scores[n]]) for n in scores.keys()})
    
    # Sort and return list
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)

In [38]:
#### Function check
test_naiveql = naive_ql_search("report", index_set=1)[:5]
print(f"Naive QL Results:")
print_results(test_naiveql)
####

Naive QL Results:
Rank 0(0.2): A Report Writer For COBOL...
Rank 1(0.2): A CRT Report Generating System...
Rank 2(0.17): Preliminary Report-International Algebraic Languag...
Rank 3(0.17): Supplement to the ALGOL 60 Report...
Rank 4(0.14): ALGOL Sub-Committee Report - Extensions...


In [39]:
#### Function check
test_naiveql = naive_ql_search("report " * 10, index_set=1)[:5]
print(f"Naive QL Results:")
print_results(test_naiveql)
####

Naive QL Results:
Rank 0(1e-07): A Report Writer For COBOL...
Rank 1(1e-07): A CRT Report Generating System...
Rank 2(1.7e-08): Preliminary Report-International Algebraic Languag...
Rank 3(1.7e-08): Supplement to the ALGOL 60 Report...
Rank 4(3.5e-09): ALGOL Sub-Committee Report - Extensions...


In [40]:
#### Function check
test_naiveql = naive_ql_search("report ALGOL Supplement", index_set=1)[:5]
print(f"Naive QL Results:")
print_results(test_naiveql)
####

Naive QL Results:
Rank 0(0.0046): Supplement to the ALGOL 60 Report...
Rank 1(0.0): Preliminary Report-International Algebraic Languag...
Rank 2(0.0): Extraction of Roots by Repeated Subtractions for D...
Rank 3(0.0): Techniques Department on Matrix Program Schemes...
Rank 4(0.0): Glossary of Computer Engineering and Programming T...


In [41]:
#### Please do not change this. This cell is used for grading.

In [42]:
#### Please do not change this. This cell is used for grading.

In [43]:
#### Please do not change this. This cell is used for grading.

In [44]:
#### Please do not change this. This cell is used for grading.

---
#### 3.3.2 QL (20 points)
Now, let's implement a QL model that handles the issues with the naive version. In particular, you will implement a QL model with Jelinek-Mercer Smoothing. That means an interpolated score is computed per word - one term is the same as the previous naive version, and the second term comes from a unigram language model. In addition, you should accumulate the scores by summing the **log** (smoothed) probability which leads to better numerical stability.

In [45]:
# TODO: Implement this! (20 points)

def ql_search(query, index_set):
    """
        Perform a search over all documents with the given query using a QL model 
        with Jelinek-Mercer Smoothing (set smoothing=0.1). 
        
        
        Note #1: You have to use the `get_index` (and get_doc_lengths) function created in the previous cells
        Note #2: You might have to create some variables beforehand and use them in this function
        
        
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    
    index = get_index(index_set)
    doc_lengths = get_doc_lengths(index_set)
    processed_query = preprocess_query(query, index_set)
    
    # Collection length
    cl = sum(doc_lengths.values())
    
    # Set smoothing
    smoothing = 0.1
    
    # Make a set of all involved documents
    documents = set([z[0] for z in [y for x in index.values() for y in x]])
    
    # Specifiy defaultdict
    scores = defaultdict(list)
    
    # For each term in query
    for term in processed_query:
        
        # If term does not exist in vocab assign 0
        if term not in index:
            continue
            
        # Collect al doc_ids from documents in which term occurs
        in_docs = [x[0] for x in index[term]]
            
        # Collection frequency of term
        cft = sum([x[1] for x in index[term]])
        
        # Second term of score, as this term is indep. of document
        colfrac = (1-smoothing)*cft/cl
        
        # For every document
        for document in documents:
            
            # If term occurs in this document append probability
            if document in in_docs:            
                scores[document].append(np.log((smoothing*(index[term][in_docs.index(document)][1]/doc_lengths[document]))
                                       + colfrac))
                
            # If term does not occur in document, append second part of equation
            else:
                scores[document].append(np.log(colfrac))
    
    # Score equals the logsum of all probabilities
    # Note: log(𝑎)+log(𝑏)=log(𝑎𝑏)
    scores.update({n: np.sum([float(y) for y in scores[n]]) for n in scores.keys()})
    
    # Sort and return list
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)

In [46]:
#### Function check
test_ql_results = ql_search("report", index_set=1)[:5]
print_results(test_ql_results)
print()
test_ql_results_long = ql_search("report " * 10, index_set=1)[:5]
print_results(test_ql_results_long)
####

Rank 0(-3.9): A Report Writer For COBOL...
Rank 1(-3.9): A CRT Report Generating System...
Rank 2(-4.1): Preliminary Report-International Algebraic Languag...
Rank 3(-4.1): Supplement to the ALGOL 60 Report...
Rank 4(-4.2): ALGOL Sub-Committee Report - Extensions...

Rank 0(-3.9e+01): A Report Writer For COBOL...
Rank 1(-3.9e+01): A CRT Report Generating System...
Rank 2(-4.1e+01): Preliminary Report-International Algebraic Languag...
Rank 3(-4.1e+01): Supplement to the ALGOL 60 Report...
Rank 4(-4.2e+01): ALGOL Sub-Committee Report - Extensions...


In [47]:
#### Function check
test_ql_results_long = ql_search("report ALGOL Supplement", index_set=1)[:5]
print_results(test_ql_results_long)
####

Rank 0(-1.2e+01): Supplement to the ALGOL 60 Report...
Rank 1(-2e+01): ALGOL Sub-Committee Report - Extensions...
Rank 2(-2e+01): Report on the Algorithmic Language ALGOL 60...
Rank 3(-2e+01): Report on SUBSET ALGOL 60 (IFIP)...
Rank 4(-2.1e+01): Report on Input-Output Procedures for ALGOL 60 (IF...


In [48]:
#### Please do not change this. This cell is used for grading.

In [49]:
#### Please do not change this. This cell is used for grading.

In [50]:
#### Please do not change this. This cell is used for grading.

In [51]:
#### Please do not change this. This cell is used for grading.

--- 

### Section 3.4: BM25 (20 points) <a class="anchor" id="bm25"></a>

In this section, we will implement the BM25 scoring function. 


In [52]:
# TODO: Implement this! (20 points)
def bm25_search(query, index_set, k_1=1.5, b=0.75):
    
    """
        Perform a search over all documents with the given query using BM25. Use k_1 = 1.5 and b = 0.75
        Note #1: You have to use the `get_index` (and `get_doc_lengths`) function created in the previous cells
        Note #2: You might have to create some variables beforehand and use them in this function
        
        Input: 
            query - a (unprocessed) query
            index_set - the index to use
        Output: a list of (document_id, score), sorted in descending relevance to the given query 
    """
    
    index = get_index(index_set)
    df = get_df(index_set)
    doc_lengths = get_doc_lengths(index_set)
    processed_query = preprocess_query(query, index_set)
    documents = set([z[0] for z in [y for x in index.values() for y in x]])
    
    scores = dict({key: 0 for key in documents})
    N = len(doc_lengths)
    avdl = sum(doc_lengths.values()) / N
    for q in processed_query:
        if q not in index:
            continue
        inv_list = index[q]
        n = len(inv_list)
        
        for doc_id, f in inv_list:     
            dl = doc_lengths[doc_id]
            K = k_1*((1-b) + b*dl/avdl)
            score = np.log((N / df[q]) * (((k_1+1)*f) / (K+f)))
            scores[doc_id] += score
    
    return [(d, float(s)) for d, s in sorted(scores.items(), key=lambda t: t[1], reverse=True)]

In [53]:
#### Function check
test_bm25_results = bm25_search("report", index_set=1)[:5]
print_results(test_bm25_results)
####

Rank 0(4.4): A Report Writer For COBOL...
Rank 1(4.4): A CRT Report Generating System...
Rank 2(4.4): Preliminary Report-International Algebraic Languag...
Rank 3(4.4): Supplement to the ALGOL 60 Report...
Rank 4(4.4): ALGOL Sub-Committee Report - Extensions...


In [54]:
#### Please do not change this. This cell is used for grading.

In [55]:
#### Please do not change this. This cell is used for grading.

In [56]:
#### Please do not change this. This cell is used for grading.

In [57]:
#### Please do not change this. This cell is used for grading.


---

### 3.5. Test Your Functions

The widget below allows you to play with the search functions you've written so far. Use this to test your search functions and ensure that they work as expected.

In [58]:
#### Highlighter function
# class for results
ResultRow = namedtuple("ResultRow", ["doc_id", "snippet", "score"])
# doc_id -> doc
docs_by_id = dict((d[0], d[1]) for d in docs)

def highlight_text(document, query, tol=17):
    import re
    tokens = tokenize(query)
    regex = "|".join(f"(\\b{t}\\b)" for t in tokens)
    regex = re.compile(regex, flags=re.IGNORECASE)
    output = ""
    i = 0
    for m in regex.finditer(document):
        start_idx = max(0, m.start() - tol)
        end_idx = min(len(document), m.end() + tol)
        output += "".join(["...",
                        document[start_idx:m.start()],
                        "<strong>",
                        document[m.start():m.end()],
                        "</strong>",
                        document[m.end():end_idx],
                        "..."])
    return output.replace("\n", " ")


def make_results(query, search_fn, index_set):
    results = []
    for doc_id, score in search_fn(query, index_set):
        highlight = highlight_text(docs_by_id[doc_id], query)
        if len(highlight.strip()) == 0:
            highlight = docs_by_id[doc_id]
        results.append(ResultRow(doc_id, highlight, score))
    return results
####

In [59]:
# TODO: Set this to the function you want to test!
# this function should take in a query (string)
# and return a sorted list of (doc_id, score) 
# with the most relevant document in the first position

search_fn = bm25_search
index_set = 1

text = widgets.Text(description="Search Bar", width=200)
display(text)

def handle_submit(sender):
    print(f"Searching for: '{sender.value}'")
    
    results = make_results(sender.value, search_fn, index_set)
    
    # display only the top 5
    results = results[:5]
    
    body = ""
    for idx, r in enumerate(results):
        body += f"<li>Document #{r.doc_id}({r.score}): {r.snippet}</li>"
    display(HTML(f"<ul>{body}</ul>"))
    

text.on_submit(handle_submit)

Text(value='', description='Search Bar')

---

## Section 4: Evaluation (40 points) <a class="anchor" id="evaluation"></a>

[Back to Part 1](#part1)

Before we jump in and implement an algorithm for retrieval, we first have to learn how to evaluate such a system. In particular, we will work with offline evaluation metrics. These metrics are computed on a dataset with known relevance judgements.

Implement the following evaluation metrics. 

1. Precision (7 points)
2. Recall (7 points)
3. Mean Average Precision (12 points)
4. Expected Reciprocal Rank (12 points)

---
### 4.1 Read relevance labels (2 points)

Let's take a look at the `qrels.text` file, which contains the ground truth relevance scores. The relevance labels for CACM are binary - either 0 or 1. 


In [60]:
!head ./datasets/qrels.text

01 1410  0 0
01 1572  0 0
01 1605  0 0
01 2020  0 0
01 2358  0 0
02 2434  0 0
02 2863  0 0
02 3078  0 0
03 1134  0 0
03 1613  0 0


---
**Implementation (2 points):**
The first column is the query_id and the second column is the document_id. You can safely ignore the 3rd and 4th columns. Write a function to read in the file:

In [61]:
# TODO: Implement this! (2 points)
def read_qrels(root_folder = "./datasets/"):
    """
        Reads the qrels.text file. 
        Output: A dictionary: query_id -> [list of relevant documents]
    """
    
    path = root_folder + "qrels.text"
    queries = {}
    
    # Open and split document into lines
    with open(path, 'r') as file:
        q_list = file.read().splitlines()
    
    # Save each query and add relevant documents
    for q in q_list:
        query_id, document_id, _, _ = [int(i) for i in q.split()]
        if query_id not in queries:
            queries[query_id] = []
        queries[query_id].append(document_id)
    
    return queries

In [62]:
#### Function check
qrels = read_qrels()

assert len(qrels) == 52, "There should be 52 queries with relevance judgements"
assert sum(len(j) for j in qrels.values()) == 796, "There should be a total of 796 Relevance Judgements"
####

---
**Note:** For a given query `query_id`, you can assume that documents *not* in `qrels[query_id]` are not relevant to `query_id`. 


---
### 4.2 Precision (7 points)
Implement the `precision@k` metric:

In [63]:
# TODO: Implement this! (7 points)
def precision_k(results, relevant_docs, k):
    """
        Compute Precision@K
        Input: 
            results: A sorted list of 2-tuples (document_id, score), 
                    with the most relevant document in the first position
            relevant_docs: A set of relevant documents. 
            k: the cut-off
        Output: Precision@K
    """
    results = set([d[0] for d in results[:k]])
    relevant_docs = set(relevant_docs)
    TP = relevant_docs.intersection(results)
    return len(TP) / k

In [64]:
#### Function check
qid = queries[0][0]
qtext = queries[0][1]
print(f'query:{qtext}')
results = bm25_search(qtext, 2)
precision = precision_k(results, qrels[qid], 10)
print(f'precision@10 = {precision}')
####

query: What articles exist which deal with TSS (Time Sharing System), an operating system for IBM computers?
precision@10 = 0.1


---
### 4.3 Recall (7 points)
Implement the `recall@k` metric:

In [65]:
# TODO: Implement this! (7 points)
def recall_k(results, relevant_docs, k):
    """
        Compute Recall@K
        Input: 
            results: A sorted list of 2-tuples (document_id, score), with the most relevant document in the first position
            relevant_docs: A set of relevant documents. 
            k: the cut-off
        Output: Recall@K
    """
    results = set([d[0] for d in results[:k]])
    relevant_docs = set(relevant_docs)
    TP = relevant_docs.intersection(results)
    return len(TP) / len(relevant_docs)

In [66]:
#### Function check
qid = queries[10][0]
qtext = queries[10][1]
print(f'query:{qtext}')
results = bm25_search(qtext, 2)
recall = recall_k(results, qrels[qid], 10)
print(f'recall@10 = {recall}')
####

query: SETL, Very High Level Languages
recall@10 = 0.2631578947368421


---
### 4.4 Mean Average Precision (12 points)
Implement the `map` metric:

In [67]:
# TODO: Implement this! (12 points)
def average_precision(results, relevant_docs):
    """
        Compute Average Precision (for a single query - the results are 
        averaged across queries to get MAP in the next few cells)
        Hint: You can use the recall_k and precision_k functions here!
        Input: 
            results: A sorted list of 2-tuples (document_id, score), with the most 
                    relevant document in the first position
            relevant_docs: A set of relevant documents. 
        Output: Average Precision
    """
    ap = 0
    found = 0
    for i in range(len(results)):
        # Compute precisions at relevant documents
        if results[i][0] in relevant_docs:
            ap += precision_k(results, relevant_docs, i+1)
            found += 1
        # Stop early in case we already found all relevant documents
        if found > len(relevant_docs):
            break

    return ap / len(relevant_docs)

In [68]:
#### Function check
qid = queries[20][0]
qtext = queries[20][1]
print(f'query:{qtext}')
results = bm25_search(qtext, 2)
mean_ap = average_precision(results, qrels[qid])
print(f'MAP = {mean_ap}')
####

query: computational complexity, intractability, class-complete reductions, algorithms and efficiency
MAP = 0.23598489537146516


---
### 4.5 Expected Reciprocal Rank (12 points)
Implement the `err` metric:

In [69]:
# TODO: Implement this! (12 points)
def err(results, relevant_docs):
    """
        Compute the expected reciprocal rank.
        Hint: https://dl.acm.org/doi/pdf/10.1145/1645953.1646033?download=true
        Input: 
            results: A sorted list of 2-tuples (document_id, score), with the most 
                    relevant document in the first position
            relevant_docs: A set of relevant documents. 
        Output: ERR
        
    """
    ERR = 0
    # Relevance probabilities are constant because of binary grading
    # 0.5 for relevant documents and 0 for non-relevant ones
    rel_prob = 0.5
    rel_count = 0
    for r in range(len(results)):
        doc_id = results[r][0]
        # We only have to compute for relevant documents as
        # non-relevant documents zero out the product
        if doc_id in relevant_docs:
            rel_count += 1
            # (1-0) = 1, so non-relevant documents don't matter
            # (1-0.5) = 0.5, so all relevant documents worth the same as the r-th one
            ERR += np.power(rel_prob, rel_count) / (r+1)
        
    return ERR

In [70]:
#### Function check
qid = queries[30][0]
qtext = queries[30][1]
print(f'query:{qtext}')
results = bm25_search(qtext, 2)
ERR = err(results, qrels[qid])
print(f'ERR = {ERR}')
####

query: I'd like to find articles describing the use of singular value decomposition in digital image processing.  Applications include finding approximations to the original image and restoring images that are subject to noise. An article on the subject is H.C. Andrews and C.L. Patterson "Outer product expansions and their uses in digital image processing", American Mathematical Monthly, vol. 82.
ERR = 0.2916666666666667


---
### 4.6 Evaluate Search Functions

Let's define some metrics@k using [partial functions](https://docs.python.org/3/library/functools.html#functools.partial)

In [71]:
#### metrics@k functions

recall_at_1 = partial(recall_k, k=1)
recall_at_5 = partial(recall_k, k=5)
recall_at_10 = partial(recall_k, k=10)
precision_at_1 = partial(precision_k, k=1)
precision_at_5 = partial(precision_k, k=5)
precision_at_10 = partial(precision_k, k=10)


list_of_metrics = [
    ("ERR", err),
    ("MAP", average_precision),
    ("Recall@1",recall_at_1),
    ("Recall@5", recall_at_5),
    ("Recall@10", recall_at_10),
    ("Precision@1", precision_at_1),
    ("Precision@5", precision_at_5),
    ("Precision@10", precision_at_10)]
####

---

The following function evaluates a `search_fn` using the `metric_fn`. Note that the final number is averaged over all the queries

In [72]:
#### Evaluate a search function

list_of_search_fns = [
    ("BOW", bow_search),
    ("TF-IDF", tfidf_search),
    ("NaiveQL", naive_ql_search),
    ("QL", ql_search),
    ("BM25", bm25_search)
]

def evaluate_search_fn(search_fn, metric_fns, index_set=None):
    # build a dict query_id -> query 
    queries_by_id = dict((q[0], q[1]) for q in queries)
    
    metrics = {}
    for metric, metric_fn in metric_fns:
        metrics[metric] = np.zeros(len(qrels), dtype=np.float32)
    
    for i, (query_id, relevant_docs) in enumerate(qrels.items()):
        query = queries_by_id[query_id]
        if index_set:
            results = search_fn(query, index_set)
        else:
            results = search_fn(query)
        
        for metric, metric_fn in metric_fns:
            metrics[metric][i] = metric_fn(results, relevant_docs)

    
    
    final_dict = {}
    for metric, metric_vals in metrics.items():
        final_dict[metric] = metric_vals.mean()
    
    return final_dict
####

## Section 5: Analysis (30 points) <a class="anchor" id="analysis"></a>

[Back to Part 1](#part1)

In the final section of Part1, we will compare the different term-based IR algorithms and different preprocessing configurations and analyze their advantages and disadvantages.

### Section 5.1: Plot (20 points)

First, gather the results. The results should consider the index set, the different search functions and different metrics. Plot the results in bar charts, per metric, with clear labels.

**Rubric:**
- Each Metric is plotted: 7 points
- Each Method is plotted: 7 points
- Clear titles, x label, y labels and legends (if applicable): 6 points

In [73]:
# YOUR CODE HERE
# raise NotImplementedError()

---
### Section 5.2: Summary (10 points)
Write a summary of what you observe in the results.
Your summary should compare results across the 2 indices and the methods being used. State what you expected to see in the results, followed by either supporting evidence *or* justify why the results did not support your expectations.      

Write your answer here!

---
---
# Part 2: Semantic-based Matching (85 points) <a class="anchor" id="part2"></a>

[Back to top](#top)

We will now experiment with methods that go beyond lexical methods like TF-IDF, which operate at the word level and are high dimensional and sparse, and look at methods which constructs low dimensional dense representations of queries and documents. 

Since these low-dimensional methods have a higher time complexity, they are typically used in conjunction with methods like BM-25. That is, instead of searching through potentially million documents to find matches using low dimensional vectors, a list of K documents are retrieved using BM25, and then **re-ranked** using the other method. This is the method that is going to be applied in the following exercises. 

LSI/LDA takes documents that are similar on a semantic level - for instance, if they are describing the same topic - and projects them into nearby vectors, despite having low lexical overlap.

In this assignment, you will use `gensim` to create LSI/LDA models and use them in re-ranking. 

**Note**: The following exercises only uses `doc_repr_2` and `config_2`

Table of contents:
- [Section 6: LSI](#lsi) (15 points)
- [Section 7: LDA](#lda) (10 points)
- [Section 8: Word2Vec/Doc2Vec](#2vec) (20 points)
- [Section 8: Re-ranking](#reranking) (10 points)
- [Section 9: Re-ranking Evaluation](#reranking_eval) (30 points)

---
## Section 6: Latent Semantic Indexing (LSI) (15 points) <a class="anchor" id="lsi"></a>

[Back to Part 2](#part2)

LSI is one of the methods to embed the queries and documents into vectors. It is based on a method similar to Principal Component Analysis (PCA) for obtaining a dense concept matrix out of the sparse term-document matrix.

See [wikipedia](https://en.wikipedia.org/wiki/Latent_semantic_analysis), particularly [#Mathematics_of_LSI](https://en.wikipedia.org/wiki/Latent_semantic_analysis#Mathematics_of_LSI).

In [74]:
from gensim.corpora import Dictionary
from gensim.models import LdaModel, LsiModel, Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim import downloader as g_downloader
# gensim uses logging, so set it up 
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

---
### Section 6.1: Cosine Similarity (5 points)<a class="anchor" id="cosing_sim"></a>
Before we begin, let us first define our method of similarity for the LSI model, the cosine similarity:

$$\text{similarity} = \cos(\theta) = {\mathbf{A} \cdot \mathbf{B} \over \|\mathbf{A}\| \|\mathbf{B}\|} = \frac{ \sum\limits_{i=1}^{n}{A_i  B_i} }{ \sqrt{\sum\limits_{i=1}^{n}{A_i^2}}  \sqrt{\sum\limits_{i=1}^{n}{B_i^2}} }$$

Since we are using gensim, the types of vectors returned by their classes are of the form defined below (they are not just simple vectors):

In [75]:
# 1, 2, 3 are either latent dimensions (LSI), or topics (LDA)
# The second value in each tuple is a number (LSI) or a probability (LDA)  
example_vec_1 = [(1, 0.2), (2, 0.3), (3, 0.4)]
example_vec_2 = [(1, 0.2), (2, 0.7), (3, 0.4)]

---
**Implementation (2+3 points):**
Now, implement the `dot product` operation on these types of vectors and using this operator, implement the `cosine similarity` (don't forget: two functions to implement!):

In [76]:
# TODO: Implement this! (2 points)
def dot(vec_1,vec_2): 
    """
        vec_1 and vec_2 are of the form: [(int, float), (int, float), ...]
        Return the dot product of two such vectors, computed only on the floats
        You can assume that the lengths of the vectors are the same, and the dimensions are aligned 
            i.e you won't get: vec_1 = [(1, 0.2)] ; vec_2 = [(2, 0.3)] 
                                (dimensions are unaligned and lengths are different)
    """
    # YOUR CODE HERE
        
    return sum([i[1]*j[1] for i,j in zip(vec_1, vec_2)])

# TODO: Implement this! (3 points)
def cosine_sim(vec_1, vec_2):
    # YOUR CODE HERE
    
    try:
        return dot(vec_1, vec_2)/(np.linalg.norm([i[1] for i in vec_1])*np.linalg.norm([j[1] for j in vec_2]))
    except ZeroDivisionError:
        return 0

In [77]:
##### Function check
print(f'vectors: {(example_vec_1,example_vec_2)}')
print(f'dot product = {dot(example_vec_1,example_vec_2)}')
print(f'cosine similarity = {cosine_sim(example_vec_1,example_vec_2)}')
##### 

vectors: ([(1, 0.2), (2, 0.3), (3, 0.4)], [(1, 0.2), (2, 0.7), (3, 0.4)])
dot product = 0.41000000000000003
cosine similarity = 0.9165587597202866


In [78]:
#### Please do not change this. This cell is used for grading.

---
### Section 6.2: LSI Retrieval (10 points)<a class="anchor" id="lsi_retrieval"></a>
LSI retrieval is simply ranking the documents based on their cosine similarity to the query vector.
First, let's write a parent class for vector-based retrieval models:

In [79]:
class VectorSpaceRetrievalModel:
    """
        Parent class for Dense Vector Retrieval models
    """
    def __init__(self, doc_repr):
        """
            document_collection: 
                [
                    (doc_id_1, [token 1, token 2, ...]), 
                    (doc_id_2, [token 1, token 2, ....]) 
                    ...
                ]

        """
        self.doc_repr = doc_repr
        self.documents = [_[1] for _ in self.doc_repr]
        
        # construct a dictionary
        self.dictionary = Dictionary(self.documents)
        # Filter out words that occur less than 20 documents, or more than 50% of the documents.
        self.dictionary.filter_extremes(no_below=10)
        self.corpus = [self.dictionary.doc2bow(doc) for doc in self.documents]
    
        # Make a index to word dictionary.
        temp = self.dictionary[0]  # This is only to "load" the dictionary.
        self.id2word = self.dictionary.id2token
        
        # this is set by the train_model function
        self.model = None
        
        
    def vectorize_documents(self):
        """
            Returns a doc_id -> vector dictionary
        """
        vectors = {}
        for (doc_id, _), cc in zip(self.doc_repr, self.corpus):
            vectors[doc_id] = self.model[cc]
        return vectors

    def vectorize_query(self, query):
        # Note the use of config_2 here!
        query = process_text(query, **config_2)
        query_vector = self.dictionary.doc2bow(query)
        return self.model[query_vector]
    
    def train_model(self):
        """
            Trains a model and sets the 'self.model' variable. 
            Make sure to use the variables created in the __init__ method.
            e.g the variables which may be useful: {corpus, dictionary, id2word}
        """
        raise NotImplementedError()

---
**Implementation (5 points):**
Implement the `train_model` method in the following class (note that this is only one line of code in `gensim`!). Ensure that the parameters defined in the `__init__` method are not changed, and are *used in the `train_method` function*. Normally, the hyperaparameter space will be searched using grid search / other methods - in this assignment we have provided the hyperparameters for you.

The last two lines of code train an LSI model on the list of documents which have been stemmed, lower-cased and have stopwords removed. 

In [80]:
# TODO: Implement this! (5 points)
class LsiRetrievalModel(VectorSpaceRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        
        self.num_topics = 100
        self.chunksize = 2000
    
    def train_model(self):
        # YOUR CODE HERE
        
        self.model = LsiModel(self.corpus, id2word=self.id2word, num_topics=self.num_topics, chunksize=self.chunksize)

In [81]:
##### Function check
lsi = LsiRetrievalModel(doc_repr_2)
lsi.train_model()

# you can now get an LSI vector for a given query in the following way:
lsi.vectorize_query("report")
##### 

2021-02-14 16:48:17,679 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-02-14 16:48:17,762 : INFO : built Dictionary(5937 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 115970 corpus positions)
2021-02-14 16:48:17,766 : INFO : discarding 4740 tokens: [('repeat', 8), ('glossari', 7), ('inspect', 8), ('uncol', 2), ('rung', 9), ('secant', 2), ('.', 1603), ('acceler', 6), ('diverg', 3), ('induc', 9)]...
2021-02-14 16:48:17,766 : INFO : keeping 1197 tokens which were in no less than 10 and no more than 1602 (=50.0%) documents
2021-02-14 16:48:17,768 : INFO : resulting dictionary: Dictionary(1197 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...)
2021-02-14 16:48:17,815 : INFO : using serial LSI version on this node
2021-02-14 16:48:17,816 : INFO : updating model with new documents
2021-02-14 16:48:17,816 : INFO : preparing a new chunk of documents
2021-02-14 16:48:17,826 : INFO : using 100 extra sam

[(0, 0.01521721701128651),
 (1, -0.01628138333116389),
 (2, -0.00016742373977551763),
 (3, -0.0018930378826455326),
 (4, -0.009487589691334893),
 (5, -0.004634316771987595),
 (6, 0.02695853351923618),
 (7, 0.016802168894632697),
 (8, -0.03166601553338791),
 (9, -0.000777001454443961),
 (10, 0.0023438042799367852),
 (11, -0.01727653433956524),
 (12, -0.0003491837981983645),
 (13, 0.0012016687884990563),
 (14, 0.004372424732935422),
 (15, 0.005258791921621377),
 (16, 0.00480144741341728),
 (17, 0.0020854402947404673),
 (18, -0.01773519276757521),
 (19, 0.020552433878636606),
 (20, -0.009443100450220742),
 (21, -0.013196027343663146),
 (22, 0.048220544218979984),
 (23, 0.023545489837345897),
 (24, -0.011864213851772571),
 (25, -0.010757124568771795),
 (26, 0.009044177426892196),
 (27, 0.07672873262343788),
 (28, -0.06299655961621128),
 (29, 0.031815932585366656),
 (30, 0.041971058496887675),
 (31, 0.04368482751396911),
 (32, -0.07158200247166051),
 (33, 0.047973827428948526),
 (34, -0.024

\#### Please do not change this. This cell is used for grading.

---
Next, implement a basic ranking class for vector space retrieval (used for all semantic methods): 

In [82]:
# TODO: Implement this! (5 points)

class DenseRetrievalRanker:
    def __init__(self, vsrm, similarity_fn):
        """
            vsrm: instance of `VectorSpaceRetrievalModel`
            similarity_fn: function instance that takes in two vectors 
                            and returns a similarity score e.g cosine_sim defined earlier
        """
        self.vsrm = vsrm 
        self.vectorized_documents = self.vsrm.vectorize_documents()
        self.similarity_fn = similarity_fn
    
    def _compute_sim(self, query_vector):
        """
            Compute the similarity of `query_vector` to documents in 
            `self.vectorized_documents` using `self.similarity_fn`
            Returns a list of (doc_id, score) tuples
        """
        
        # YOUR CODE HERE
        
        # Calculate cosine similarity score for every document
        scores = {key: self.similarity_fn(query_vector, value) for (key, value) in self.vectorized_documents.items()}
        
        # Return list with tuples containing document_id sorted on score in descending order
        return sorted(scores.items(), key=lambda x: x[1], reverse=True)
    
    def search(self, query):
        scores = self._compute_sim(self.vsrm.vectorize_query(query))
        scores.sort(key=lambda _:-_[1])
        return scores 

In [83]:
##### Function check
drm_lsi = DenseRetrievalRanker(lsi, cosine_sim)
drm_lsi.search("report")[:5]
##### 

<ipython-input-76-56b467aaba55>:19: RuntimeWarning: invalid value encountered in double_scalars
  return dot(vec_1, vec_2)/(np.linalg.norm([i[1] for i in vec_1])*np.linalg.norm([j[1] for j in vec_2]))


[(599, 0.8057825833106537),
 (53, 0.49513982528086603),
 (1339, 0.4438209037157682),
 (2181, 0.43472938756158347),
 (196, 0.4230009364967407)]

\#### Please do not change this. This cell is used for grading.

---
Now, you can test your LSI model in the following cell: try finding queries which are lexically different to documents, but semantically similar - does LSI work well for these queries?!

In [84]:
# test your LSI model
search_fn = drm_lsi.search

text = widgets.Text(description="Search Bar", width=200)
display(text)

def make_results_2(query, search_fn):
    results = []
    for doc_id, score in search_fn(query):
        highlight = highlight_text(docs_by_id[doc_id], query)
        if len(highlight.strip()) == 0:
            highlight = docs_by_id[doc_id]
        results.append(ResultRow(doc_id, highlight, score))
    return results

def handle_submit_2(sender):
    print(f"Searching for: '{sender.value}' (SEARCH FN: {search_fn})")
    
    results = make_results_2(sender.value, search_fn)
    
    # display only the top 5
    results = results[:5]
    
    body = ""
    for idx, r in enumerate(results):
        body += f"<li>Document #{r.doc_id}({r.score}): {r.snippet}</li>"
    display(HTML(f"<ul>{body}</ul>"))
    

text.on_submit(handle_submit_2)

Text(value='', description='Search Bar')

---
## Section 7: Latent Dirichlet Allocation (LDA) (10 points) <a class="anchor" id="lda"></a>

[Back to Part 2](#part2)

The specifics of LDA is out of the scope of this assignment, but we will use the `gensim` implementation to perform search using LDA over our small document collection. The key thing to remember is that LDA, unlike LSI, outputs a topic **distribution**, not a vector. With that in mind, let's first define a similarity measure.


---
### Section 7.1: Jenson-Shannon divergence (5 points) <a class="anchor" id="js_sim"></a>

The Jenson-Shannon divergence is a symmetric and finite measure on two probability distributions (unlike the KL, which is neither). For identical distributions, the JSD is equal to 0, and since our code uses 0 as irrelevant and higher scores as relevant, we use `(1 - JSD)` as the score or 'similarity' in our setup

**Note**: the JSD is bounded to \[0,1\] only if we use log base 2. So please ensure that you're using `np.log2` instead of `np.log`

In [85]:
## TODO: Implement this! (5 points)
# Note for TA: if assert_prob = True, in part 9 numerical value error (it is not exactly 1.0000 and thus function
#errors)
def jenson_shannon_divergence(vec_1, vec_2, assert_prob=False):
    """
        Computes the Jensen-Shannon divergence between two probability distributions. 
        NOTE: DO NOT RETURN 1 - JSD here, that is handled by the next function which is already implemented! 
        The inputs are *gensim* vectors - same as the vectors for the cosine_sim function
        assert_prob is a flag that checks if the inputs are proper probability distributions 
            i.e they sum to 1 and are positive - use this to check your inputs if needed. 
                (This is optional to implement, but recommended - 
                you can the default to False to save a few ms off the runtime)
    """
    # YOUR CODE HERE
    
    if assert_prob:
        if sum([x for _, x in vec_1 if x>=0]) != 1 or sum([y for _, y in vec_2 if y>=0]) != 1:
            raise ValueError(f"Invalid distribution encountered")
 
    # Calculate M = 0.5 (P + Q)
    M = [(P[1] + Q[1])/2 for P, Q in zip(vec_1, vec_2)]
    
    # JSD = 0.5 * sum(log P - log M) + 0.5 * Q*sum(log Q - log M)
    return 0.5*sum([P[1]*(np.log2(P[1]/m)) for P, m in zip (vec_1, M)]) + 0.5*sum([Q[1]*np.log2(Q[1]/m) for Q, m in zip (vec_2, M)])
    

def jenson_shannon_sim(vec_1, vec_2, assert_prob=False):
    return 1 - jenson_shannon_divergence(vec_1, vec_2)

In [86]:
##### Function check
vec_1 = [(1, 0.3), (2, 0.4), (3, 0.3)]
vec_2 = [(1, 0.1), (2, 0.7), (3, 0.2)]
jenson_shannon_sim(vec_1, vec_2, assert_prob=True)
##### 

0.9251064410358459

---
### Section 7.2: LDA retrieval (5 points) <a class="anchor" id="lda_ret"></a>

Implement the `train_model` method in the following class (note that this is only one line of code in `gensim`!). Ensure that the parameters defined in the `__init__` method are not changed, and are *used in the `train_method` function*. You do not need to set this. Normally, the hyperaparameter space will be searched using grid search / other methods. Note that training the LDA model might take some time

The last two lines of code train an LDA model on the list of documents which have been stemmed, lower-cased and have stopwords removed. 

In [87]:
# TODO: Implement this! (5 points)
class LdaRetrievalModel(VectorSpaceRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        
        # use these parameters in the train_model method
        self.num_topics = 100
        self.chunksize = 2000
        self.passes = 20
        self.iterations = 400
        self.eval_every = 10
        # this is need to get full vectors
        self.minimum_probability=0.0
        self.alpha='auto'
        self.eta='auto'
    
    
    def train_model(self):
        # YOUR CODE HERE
        self.model = LdaModel(self.corpus, id2word=self.id2word, num_topics=self.num_topics, chunksize=self.chunksize, 
                        passes=self.passes, iterations=self.iterations, eval_every=self.eval_every, 
                        minimum_probability=self.minimum_probability, alpha=self.alpha, eta=self.eta)

In [88]:
##### Function check
lda = LdaRetrievalModel(doc_repr_2)
lda.train_model()

# you can now get an LDA vector for a given query in the following way:
lda.vectorize_query("report")
##### 

2021-02-14 16:48:19,071 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-02-14 16:48:19,154 : INFO : built Dictionary(5937 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 115970 corpus positions)
2021-02-14 16:48:19,158 : INFO : discarding 4740 tokens: [('repeat', 8), ('glossari', 7), ('inspect', 8), ('uncol', 2), ('rung', 9), ('secant', 2), ('.', 1603), ('acceler', 6), ('diverg', 3), ('induc', 9)]...
2021-02-14 16:48:19,158 : INFO : keeping 1197 tokens which were in no less than 10 and no more than 1602 (=50.0%) documents
2021-02-14 16:48:19,160 : INFO : resulting dictionary: Dictionary(1197 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...)
2021-02-14 16:48:19,205 : INFO : using autotuned alpha, starting with [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0

2021-02-14 16:48:21,693 : INFO : merging changes from 2000 documents into a model of 3204 documents
2021-02-14 16:48:21,696 : INFO : topic #29 (0.009): 0.001*"implic" + 0.001*"favor" + 0.001*"adapt" + 0.001*"polygon" + 0.001*"stage" + 0.001*"situat" + 0.001*"respons" + 0.001*"industri" + 0.001*"regress" + 0.001*"drive"
2021-02-14 16:48:21,697 : INFO : topic #51 (0.010): 0.059*"parallel" + 0.055*"goal" + 0.038*"program" + 0.037*"design" + 0.037*"tabl" + 0.037*"repetit" + 0.034*"notat" + 0.034*"process" + 0.032*"system" + 0.030*"path"
2021-02-14 16:48:21,697 : INFO : topic #62 (0.010): 0.089*"," + 0.025*"-" + 0.025*"system" + 0.019*"(" + 0.015*")" + 0.015*""" + 0.013*"comput" + 0.013*"program" + 0.012*"process" + 0.012*"problem"
2021-02-14 16:48:21,698 : INFO : topic #98 (0.011): 0.078*"," + 0.027*"-" + 0.025*"program" + 0.019*"tabl" + 0.018*"decis" + 0.015*"comput" + 0.014*"control" + 0.014*"optim" + 0.013*"time" + 0.012*"paper"
2021-02-14 16:48:21,698 : INFO : topic #50 (0.011): 0.085*

2021-02-14 16:48:23,883 : INFO : merging changes from 1204 documents into a model of 3204 documents
2021-02-14 16:48:23,887 : INFO : topic #29 (0.009): 0.001*"implic" + 0.001*"favor" + 0.001*"adapt" + 0.001*"polygon" + 0.001*"stage" + 0.001*"situat" + 0.001*"respons" + 0.001*"industri" + 0.001*"regress" + 0.001*"drive"
2021-02-14 16:48:23,887 : INFO : topic #70 (0.010): 0.070*"," + 0.057*"count" + 0.050*"usag" + 0.043*"-" + 0.034*"diagram" + 0.028*"comput" + 0.027*"composit" + 0.025*"effici" + 0.022*"machin" + 0.022*"greater"
2021-02-14 16:48:23,888 : INFO : topic #28 (0.011): 0.186*"]" + 0.072*"1" + 0.055*"approxim" + 0.052*"2" + 0.037*"5" + 0.036*"linear" + 0.033*"primari" + 0.030*"root" + 0.028*"norm" + 0.025*"/"
2021-02-14 16:48:23,888 : INFO : topic #32 (0.011): 0.298*")" + 0.297*"(" + 0.291*"algorithm" + 0.025*"partit" + 0.024*"function" + 0.015*"gener" + 0.013*"fraction" + 0.013*"elementari" + 0.008*"constraint" + 0.007*"continu"
2021-02-14 16:48:23,888 : INFO : topic #50 (0.012

2021-02-14 16:48:25,540 : INFO : topic #70 (0.010): 0.075*"," + 0.050*"count" + 0.046*"diagram" + 0.044*"-" + 0.044*"usag" + 0.034*"comput" + 0.032*"machin" + 0.028*"effici" + 0.028*"gener" + 0.026*"greater"
2021-02-14 16:48:25,540 : INFO : topic #3 (0.012): 0.402*"(" + 0.348*")" + 0.057*"algorithm" + 0.047*"1" + 0.034*"integr" + 0.015*"complet" + 0.012*"seri" + 0.011*"program" + 0.011*"ellipt" + 0.011*"fourier"
2021-02-14 16:48:25,540 : INFO : topic #50 (0.012): 0.123*"system" + 0.080*"," + 0.049*"-" + 0.038*"user" + 0.028*"data" + 0.027*"oper" + 0.025*"process" + 0.022*"program" + 0.018*"time" + 0.017*"share"
2021-02-14 16:48:25,541 : INFO : topic #32 (0.013): 0.311*"algorithm" + 0.307*")" + 0.304*"(" + 0.022*"function" + 0.014*"partit" + 0.011*"gener" + 0.009*"fraction" + 0.006*"continu" + 0.005*"elementari" + 0.005*"constraint"
2021-02-14 16:48:25,541 : INFO : topic diff=0.610432, rho=0.389191
2021-02-14 16:48:25,995 : INFO : -6.643 per-word bound, 100.0 perplexity estimate based o

2021-02-14 16:48:27,520 : INFO : topic #70 (0.010): 0.090*"," + 0.071*"count" + 0.062*"usag" + 0.045*"effici" + 0.040*"diagram" + 0.038*"machin" + 0.036*"-" + 0.035*"comput" + 0.035*"greater" + 0.030*"composit"
2021-02-14 16:48:27,521 : INFO : topic #50 (0.013): 0.134*"system" + 0.083*"," + 0.053*"-" + 0.045*"user" + 0.028*"oper" + 0.024*"process" + 0.021*"data" + 0.020*"time" + 0.020*"share" + 0.019*"program"
2021-02-14 16:48:27,521 : INFO : topic #3 (0.013): 0.442*"(" + 0.373*")" + 0.049*"1" + 0.032*"algorithm" + 0.025*"integr" + 0.013*"complet" + 0.009*"fourier" + 0.008*"program" + 0.007*"problem" + 0.007*"machin"
2021-02-14 16:48:27,521 : INFO : topic #32 (0.015): 0.345*"algorithm" + 0.298*")" + 0.295*"(" + 0.018*"function" + 0.010*"fraction" + 0.008*"gener" + 0.008*"partit" + 0.005*"continu" + 0.003*"elementari" + 0.003*"constraint"
2021-02-14 16:48:27,522 : INFO : topic diff=0.641410, rho=0.362690
2021-02-14 16:48:27,526 : INFO : PROGRESS: pass 6, at document #2000/3204
2021-02-1

...

2021-02-14 16:48:37,336 : INFO : topic #32 (0.030): 0.402*"algorithm" + 0.281*")" + 0.281*"(" + 0.016*"function" + 0.009*"fraction" + 0.002*"constraint" + 0.002*"continu" + 0.000*"elementari" + 0.000*"gener" + 0.000*"partit"
2021-02-14 16:48:37,336 : INFO : topic diff=0.212897, rho=0.245426
2021-02-14 16:48:37,340 : INFO : PROGRESS: pass 15, at document #2000/3204
2021-02-14 16:48:37,696 : INFO : optimized alpha [0.012033407, 0.01702741, 0.01271928, 0.023439884, 0.011253469, 0.011715653, 0.014205956, 0.011351942, 0.011886145, 0.012656055, 0.013396495, 0.014688869, 0.011916894, 0.011577021, 0.01243294, 0.013064827, 0.012557058, 0.013440116, 0.013515307, 0.011172115, 0.011398361, 0.01110584, 0.012611502, 0.013018106, 0.012833534, 0.013590083, 0.012792925, 0.011655671, 0.01616069, 0.0072741956, 0.013154003, 0.013472893, 0.030787146, 0.01246892, 0.017360508, 0.01302019, 0.014048896, 0.011028777, 0.0146623235, 0.012947171, 0.013314447, 0.014631912, 0.011889737, 0.01261168, 0.0124852825, 0.0

2021-02-14 16:48:38,746 : INFO : topic #32 (0.033): 0.394*"algorithm" + 0.286*")" + 0.285*"(" + 0.017*"function" + 0.008*"fraction" + 0.003*"constraint" + 0.002*"continu" + 0.000*"elementari" + 0.000*"gener" + 0.000*"partit"
2021-02-14 16:48:38,746 : INFO : topic diff=0.174917, rho=0.231857
2021-02-14 16:48:39,152 : INFO : -6.448 per-word bound, 87.3 perplexity estimate based on a held-out corpus of 1204 documents with 49783 words
2021-02-14 16:48:39,152 : INFO : PROGRESS: pass 16, at document #3204/3204
2021-02-14 16:48:39,426 : INFO : optimized alpha [0.012264544, 0.017513126, 0.013070884, 0.025231304, 0.011461127, 0.011971188, 0.014774343, 0.011578425, 0.012163327, 0.012962316, 0.0138536645, 0.015247512, 0.012129427, 0.011868632, 0.01272113, 0.013487388, 0.012883274, 0.013837105, 0.013940611, 0.011370793, 0.011547129, 0.011358795, 0.012984262, 0.013440345, 0.013335148, 0.01401094, 0.013145979, 0.011940251, 0.01678751, 0.0071589164, 0.013526918, 0.01387629, 0.033318244, 0.012824603, 

2021-02-14 16:48:40,471 : INFO : topic #32 (0.035): 0.413*"algorithm" + 0.278*"(" + 0.275*")" + 0.016*"function" + 0.009*"fraction" + 0.002*"constraint" + 0.001*"continu" + 0.000*"elementari" + 0.000*"gener" + 0.000*"partit"
2021-02-14 16:48:40,472 : INFO : topic diff=0.166831, rho=0.225865
2021-02-14 16:48:40,476 : INFO : PROGRESS: pass 18, at document #2000/3204
2021-02-14 16:48:40,819 : INFO : optimized alpha [0.012402036, 0.018297482, 0.013327478, 0.026831018, 0.011714013, 0.0121034095, 0.015260142, 0.011697058, 0.012364847, 0.013243141, 0.014157191, 0.015616047, 0.012293908, 0.012069176, 0.012957058, 0.013831961, 0.013089779, 0.014222518, 0.01438117, 0.011498958, 0.011614603, 0.01148205, 0.013284218, 0.013805414, 0.0136796, 0.014304528, 0.0133538, 0.012113859, 0.017125973, 0.007061415, 0.013693882, 0.014061107, 0.03626763, 0.0130453585, 0.019141296, 0.013536164, 0.014974489, 0.011410885, 0.01575294, 0.013625327, 0.014018492, 0.015668774, 0.012411959, 0.01344253, 0.013072617, 0.010

2021-02-14 16:48:41,857 : INFO : topic diff=0.142531, rho=0.215156
2021-02-14 16:48:42,266 : INFO : -6.431 per-word bound, 86.3 perplexity estimate based on a held-out corpus of 1204 documents with 49783 words
2021-02-14 16:48:42,266 : INFO : PROGRESS: pass 19, at document #3204/3204
2021-02-14 16:48:42,541 : INFO : optimized alpha [0.012624699, 0.018679753, 0.013672837, 0.028666051, 0.011941612, 0.012349929, 0.015816882, 0.011912032, 0.012642595, 0.013521295, 0.014618449, 0.016168028, 0.012499334, 0.012376997, 0.013239897, 0.014255643, 0.013398854, 0.014608453, 0.014809919, 0.011698155, 0.011785488, 0.011762773, 0.013656479, 0.014241295, 0.014324183, 0.014724323, 0.013699143, 0.012398189, 0.017719148, 0.0069629746, 0.01404448, 0.014448626, 0.038913056, 0.013378705, 0.020053064, 0.013831215, 0.0155308535, 0.011658955, 0.016373709, 0.014009251, 0.0144201135, 0.016166054, 0.012736967, 0.013850946, 0.013455513, 0.010936579, 0.014265984, 0.012734499, 0.012709615, 0.013774277, 0.018600484, 

[(0, 0.005158832),
 (1, 0.0076331096),
 (2, 0.0055871326),
 (3, 0.011713811),
 (4, 0.004879702),
 (5, 0.0050465525),
 (6, 0.0064632543),
 (7, 0.0048676147),
 (8, 0.005166145),
 (9, 0.005525208),
 (10, 0.0059735384),
 (11, 0.006606743),
 (12, 0.0051076044),
 (13, 0.0050576134),
 (14, 0.0054102205),
 (15, 0.005825285),
 (16, 0.005475175),
 (17, 0.0059694536),
 (18, 0.0060517783),
 (19, 0.004780218),
 (20, 0.004815905),
 (21, 0.004806623),
 (22, 0.0055804485),
 (23, 0.0058194217),
 (24, 0.005853292),
 (25, 0.0060168016),
 (26, 0.0055978824),
 (27, 0.0050662733),
 (28, 0.007240577),
 (29, 0.002845281),
 (30, 0.0057389974),
 (31, 0.0059041437),
 (32, 0.015901046),
 (33, 0.0054669417),
 (34, 0.008194285),
 (35, 0.0056518507),
 (36, 0.0063463743),
 (37, 0.0047642),
 (38, 0.0066907904),
 (39, 0.005724602),
 (40, 0.0058924924),
 (41, 0.006605936),
 (42, 0.005204708),
 (43, 0.0056599136),
 (44, 0.005498328),
 (45, 0.004469015),
 (46, 0.0058295107),
 (47, 0.0052036997),
 (48, 0.005193531),
 (49, 

\#### Please do not change this. This cell is used for grading.

---
Now we can use the `DenseRetrievalModel` class to obtain an LDA search function.
You can test your LDA model in the following cell: Try finding queries which are lexically different to documents, but semantically similar - does LDA work well for these queries?!

In [89]:
drm_lda = DenseRetrievalRanker(lda, jenson_shannon_sim)

# test your LDA model
search_fn = drm_lda.search

text = widgets.Text(description="Search Bar", width=200)
display(text)


text.on_submit(handle_submit_2)

Text(value='', description='Search Bar')

## Section 8: Word2Vec/Doc2Vec (20 points) <a class="anchor" id="2vec"></a>

[Back to Part 2](#part2)

We will implement two other methods here, the Word2Vec model and the Doc2Vec model, also using `gensim`. Word2Vec creates representations of words, not documents, so the word level vectors need to be aggregated to obtain a representation for the document. Here, we will simply take the mean of the vectors. 


A drawback of these models is that they need a lot of training data. Our dataset is tiny, so in addition to using a model trained on the data, we will also use a pre-trained model for Word2Vec (this will be automatically downloaded).     

*Note*:
1. The code in vectorize_documents / vectorize_query should return gensim-like vectors i.e `[(dim, val), .. (dim, val)]`. 
2. For Word2Vec: You should also handle the following two cases: (a) A word in the query is not present in the vocabulary of the model and (b) none of the words in the query are present in the model - you can return 0 scores for all documents in this case. For either of these, you can check if a `word` is present in the vocab by using `word in self.model`


In [90]:
class W2VRetrievalModel(VectorSpaceRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        
        # the dimensionality of the vectors
        self.size = 100 
        self.min_count = 1
    
    def train_model(self):
        """
        Trains the W2V model
        """
        # YOUR CODE HERE
        self.model = Word2Vec(self.documents)
        
    def vectorize_documents(self):
        """
            Returns a doc_id -> vector dictionary
        """
        
        # YOUR CODE HERE
        doc_dict = {}
        for doc_id, content in self.doc_repr:
            doc_dict[doc_id] = np.zeros(self.size)
            for token in content:
                if token in self.model:
                    doc_dict[doc_id] += self.model[token]
            values = list(doc_dict[doc_id] / len(content))
            doc_dict[doc_id] = [(dim+1, values[dim]) for dim in range(len(values))]

        return doc_dict
        
    def vectorize_query(self, query):
        """
        Vectorizes the query using the W2V model
        """
        query = process_text(query, **config_2)
        # YOUR CODE HERE
        vector = np.zeros(self.size)
        for token in query:
            if token in self.model:
                vector += self.model[token]
        vector = vector / len(query)
        return [(dim+1, vector[dim]) for dim in range(len(vector))]
    
    
class W2VPretrainedRetrievalModel(W2VRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        self.model_name = "word2vec-google-news-300"
        self.size = 300
    
    def train_model(self):
        """
        Loads the pretrained model
        """
        self.model = g_downloader.load(self.model_name)

w2v = W2VRetrievalModel(doc_repr_2)
w2v.train_model()

# you can now get a W2V vector for a given query in the following way:
w2v.vectorize_query("report")

2021-02-14 16:48:43,694 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-02-14 16:48:43,777 : INFO : built Dictionary(5937 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 115970 corpus positions)
2021-02-14 16:48:43,781 : INFO : discarding 4740 tokens: [('repeat', 8), ('glossari', 7), ('inspect', 8), ('uncol', 2), ('rung', 9), ('secant', 2), ('.', 1603), ('acceler', 6), ('diverg', 3), ('induc', 9)]...
2021-02-14 16:48:43,781 : INFO : keeping 1197 tokens which were in no less than 10 and no more than 1602 (=50.0%) documents
2021-02-14 16:48:43,783 : INFO : resulting dictionary: Dictionary(1197 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...)
2021-02-14 16:48:43,831 : INFO : collecting all words and their counts
2021-02-14 16:48:43,832 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-02-14 16:48:43,845 : INFO : collected 5937 word types from a corpus of 115970 raw w

[(1, 0.5186378359794617),
 (2, 0.4106886684894562),
 (3, -0.28466159105300903),
 (4, 0.16464239358901978),
 (5, -0.7143939137458801),
 (6, -0.3429802358150482),
 (7, 0.001524992985650897),
 (8, 0.3702363967895508),
 (9, 0.3202510476112366),
 (10, 0.524116575717926),
 (11, -0.2719111442565918),
 (12, 0.025970863178372383),
 (13, 0.06032619625329971),
 (14, -0.05306322127580643),
 (15, -0.07226842641830444),
 (16, 0.004693465773016214),
 (17, -0.13017289340496063),
 (18, 0.3717740476131439),
 (19, 0.33895403146743774),
 (20, -0.23581135272979736),
 (21, -0.016321903094649315),
 (22, -0.10351818799972534),
 (23, -0.24264360964298248),
 (24, -0.1955231875181198),
 (25, -0.2334461212158203),
 (26, -0.42841416597366333),
 (27, -0.4016002416610718),
 (28, -0.09704328328371048),
 (29, 0.23361480236053467),
 (30, -0.16590815782546997),
 (31, -0.4380455017089844),
 (32, -0.5029312968254089),
 (33, -0.05854091793298721),
 (34, 0.30974021553993225),
 (35, 0.0981430858373642),
 (36, 0.4135043919086

In [91]:
assert len(w2v.vectorize_query("report")) == 100
assert len(w2v.vectorize_query("this is a sentence that is not mellifluous")) == 100


<ipython-input-90-58fe68e8332b>:41: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if token in self.model:
<ipython-input-90-58fe68e8332b>:42: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vector += self.model[token]


\#### Please do not change this. This cell is used for grading.

In [92]:
w2v_pretrained = W2VPretrainedRetrievalModel(doc_repr_2)
w2v_pretrained.train_model()

# you can now get an W2V vector for a given query in the following way:
w2v_pretrained.vectorize_query("report")

2021-02-14 16:48:44,389 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-02-14 16:48:44,471 : INFO : built Dictionary(5937 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 115970 corpus positions)
2021-02-14 16:48:44,475 : INFO : discarding 4740 tokens: [('repeat', 8), ('glossari', 7), ('inspect', 8), ('uncol', 2), ('rung', 9), ('secant', 2), ('.', 1603), ('acceler', 6), ('diverg', 3), ('induc', 9)]...
2021-02-14 16:48:44,476 : INFO : keeping 1197 tokens which were in no less than 10 and no more than 1602 (=50.0%) documents
2021-02-14 16:48:44,477 : INFO : resulting dictionary: Dictionary(1197 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...)
2021-02-14 16:48:44,688 : INFO : loading projection weights from /Users/charlottefelius/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz
2021-02-14 16:49:18,010 : INFO : loaded (3000000, 300) matrix from /Users/charlottefelius/gensim-data/wor

[(1, -0.142578125),
 (2, -0.1640625),
 (3, -0.09033203125),
 (4, -0.1123046875),
 (5, 0.10009765625),
 (6, -0.041259765625),
 (7, 0.048828125),
 (8, -0.13671875),
 (9, 0.1962890625),
 (10, -0.134765625),
 (11, -0.017578125),
 (12, 0.0322265625),
 (13, 0.09521484375),
 (14, -0.10595703125),
 (15, -0.169921875),
 (16, 0.041015625),
 (17, -0.263671875),
 (18, -0.006317138671875),
 (19, -0.177734375),
 (20, -0.240234375),
 (21, 0.3515625),
 (22, -0.01220703125),
 (23, -0.162109375),
 (24, -0.12060546875),
 (25, 0.043212890625),
 (26, 0.10986328125),
 (27, 0.052490234375),
 (28, 0.1787109375),
 (29, -0.1455078125),
 (30, 0.1376953125),
 (31, -0.08203125),
 (32, -0.283203125),
 (33, -0.10888671875),
 (34, -0.2890625),
 (35, 0.072265625),
 (36, -0.04736328125),
 (37, 0.040283203125),
 (38, 0.06787109375),
 (39, 0.11669921875),
 (40, 0.00083160400390625),
 (41, 0.068359375),
 (42, 0.1201171875),
 (43, -0.08837890625),
 (44, 0.337890625),
 (45, -0.044677734375),
 (46, -0.0301513671875),
 (47, 0

In [93]:
##### Function check

print(len(w2v_pretrained.vectorize_query("report")))
#####

300


In [94]:
drm_w2v = DenseRetrievalRanker(w2v, cosine_sim)

# test your LDA model
search_fn = drm_w2v.search

text = widgets.Text(description="Search Bar", width=200)
display(text)


text.on_submit(handle_submit_2)

<ipython-input-90-58fe68e8332b>:26: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if token in self.model:
<ipython-input-90-58fe68e8332b>:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  doc_dict[doc_id] += self.model[token]


Text(value='', description='Search Bar')

In [95]:
drm_w2v_pretrained = DenseRetrievalRanker(w2v_pretrained, cosine_sim)

# test your LDA model
search_fn = drm_w2v_pretrained.search

text = widgets.Text(description="Search Bar", width=200)
display(text)


text.on_submit(handle_submit_2)

Text(value='', description='Search Bar')

**Implementation (10 points):**
For Doc2Vec, you will need to create a list of `TaggedDocument` instead of using the `self.corpus` or `self.documents` variable. Use the document id as the 'tag'.
  

In [96]:
class D2VRetrievalModel(VectorSpaceRetrievalModel):
    def __init__(self, doc_repr):
        super().__init__(doc_repr)
        
        self.vector_size= 100
        self.min_count = 1
        self.epochs = 20
        
        # YOUR CODE HERE
        self.docs = []
        for doc_id, content in self.doc_repr:
            self.docs.append(TaggedDocument(words=content, tags=[doc_id]))
        
    def train_model(self):
        # YOUR CODE HERE
        self.model = Doc2Vec(
            self.docs,
            vector_size=self.vector_size,
            min_count=self.min_count,
            epochs=self.epochs
        )
    
    def vectorize_documents(self):
        """
            Returns a doc_id -> vector dictionary
        """
        # YOUR CODE HERE
        doc_dict = {}
        for doc_id, content in self.doc_repr:
            vector = self.model.infer_vector(content)
            doc_dict[doc_id] = [(dim+1, vector[dim]) for dim in range(len(vector))]
        return doc_dict

    def vectorize_query(self, query):
        # YOUR CODE HERE
        query = process_text(query, **config_2)
        vector = self.model.infer_vector(query)
        return [(dim+1, vector[dim]) for dim in range(len(vector))]
        
d2v = D2VRetrievalModel(doc_repr_2)
d2v.train_model()


# # you can now get an LSI vector for a given query in the following way:
d2v.vectorize_query("report")

2021-02-14 16:49:19,280 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-02-14 16:49:19,360 : INFO : built Dictionary(5937 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...) from 3204 documents (total 115970 corpus positions)
2021-02-14 16:49:19,364 : INFO : discarding 4740 tokens: [('repeat', 8), ('glossari', 7), ('inspect', 8), ('uncol', 2), ('rung', 9), ('secant', 2), ('.', 1603), ('acceler', 6), ('diverg', 3), ('induc', 9)]...
2021-02-14 16:49:19,365 : INFO : keeping 1197 tokens which were in no less than 10 and no more than 1602 (=50.0%) documents
2021-02-14 16:49:19,367 : INFO : resulting dictionary: Dictionary(1197 unique tokens: ['-', 'algebra', 'intern', 'languag', 'preliminari']...)
2021-02-14 16:49:19,412 : INFO : collecting all words and their counts
2021-02-14 16:49:19,413 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-02-14 16:49:19,427 : INFO : collected 5937 word types and 3205 unique tags fro

2021-02-14 16:49:23,427 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-02-14 16:49:23,427 : INFO : EPOCH - 16 : training on 115970 raw words (95474 effective words) took 0.2s, 575710 effective words/s
2021-02-14 16:49:23,581 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-02-14 16:49:23,594 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-02-14 16:49:23,595 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-02-14 16:49:23,596 : INFO : EPOCH - 17 : training on 115970 raw words (95481 effective words) took 0.2s, 572483 effective words/s
2021-02-14 16:49:23,755 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-02-14 16:49:23,762 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-02-14 16:49:23,763 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-02-14 16:49:23,763 : INFO : EPOCH - 18 : training on 115970 raw words (95489 effective w

[(1, 0.07438623),
 (2, 0.011124175),
 (3, -0.04812732),
 (4, 0.058049332),
 (5, -0.10926891),
 (6, 0.0040588104),
 (7, -0.014575478),
 (8, 0.06626364),
 (9, 0.05913307),
 (10, 0.11412705),
 (11, -0.041080523),
 (12, -0.053734004),
 (13, 0.016203258),
 (14, -0.0038779585),
 (15, -0.03175775),
 (16, 0.0025963846),
 (17, 0.003691737),
 (18, 0.091751054),
 (19, 0.05658325),
 (20, 0.023845203),
 (21, 0.026830371),
 (22, 0.03148203),
 (23, -0.02085671),
 (24, 0.0065233014),
 (25, -0.02503466),
 (26, -0.0631105),
 (27, -0.0019156778),
 (28, 0.0053187152),
 (29, 0.10496218),
 (30, -0.027715972),
 (31, -0.08729309),
 (32, -0.08273834),
 (33, -0.028941331),
 (34, 0.062591195),
 (35, 0.06087224),
 (36, 0.07047317),
 (37, -0.031160196),
 (38, -0.093346864),
 (39, 0.07367998),
 (40, 0.06277701),
 (41, 0.0930202),
 (42, -0.041853897),
 (43, 0.04830617),
 (44, -0.06246463),
 (45, 0.04001404),
 (46, -0.06463881),
 (47, 0.024967913),
 (48, -0.023919912),
 (49, -0.070957534),
 (50, 0.012147568),
 (51, -

In [97]:
#### Please do not change this. This cell is used for grading.

\#### Please do not change this. This cell is used for grading.

In [98]:
drm_d2v = DenseRetrievalRanker(d2v, cosine_sim)

# test your LDA model
search_fn = drm_d2v.search

text = widgets.Text(description="Search Bar", width=200)
display(text)


text.on_submit(handle_submit_2)

Text(value='', description='Search Bar')

---
## Section 9: Re-ranking (10 points) <a class="anchor" id="reranking"></a>

[Back to Part 2](#part2)

To motivate the re-ranking perspective (i.e retrieve with lexical method + rerank with a semantic method), let's search using semantic methods and compare it to BM25's performance, along with their runtime:


In [99]:
query = "algebraic functions"
print("BM25: ")
%timeit bm25_search(query, 2)
print("LSI: ")
%timeit drm_lsi.search(query)
print("LDA: ")
%timeit drm_lda.search(query)
print("W2V: ")
%timeit drm_w2v.search(query)
print("W2V(Pretrained): ")
%timeit drm_w2v_pretrained.search(query)
print("D2V:")
%timeit drm_d2v.search(query)

BM25: 
8.54 ms ± 665 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
LSI: 


<ipython-input-76-56b467aaba55>:19: RuntimeWarning: invalid value encountered in double_scalars
  return dot(vec_1, vec_2)/(np.linalg.norm([i[1] for i in vec_1])*np.linalg.norm([j[1] for j in vec_2]))


169 ms ± 1.42 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
LDA: 
3.85 s ± 28.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
W2V: 


<ipython-input-90-58fe68e8332b>:41: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if token in self.model:
<ipython-input-90-58fe68e8332b>:42: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vector += self.model[token]


185 ms ± 11.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
W2V(Pretrained): 
445 ms ± 14.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
D2V:
196 ms ± 4.97 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


---

**Implementation (10 points):**
Re-ranking involves retrieving a small set of documents using simple but fast methods like BM25 and then re-ranking them with the aid of semantic methods such as LDA or LSI. Implement the following class, which takes in an `initial_retrieval_fn` - the initial retrieval function and `vsrm` - an instance of the `VectorSpaceRetrievalModel` class (i.e LSI/LDA) as input. The search function should first retrieve an initial list of K documents, and then these documents are re-ranked using a semantic method. This not only makes retrieval faster, but semantic methods perform poorly when used in isolation, as you will find out.

In [100]:
# TODO: Implement this! (10 points)
class DenseRerankingModel:
    def __init__(self, initial_retrieval_fn, vsrm, similarity_fn):
        """
            initial_retrieval_fn: takes in a query and returns a list of [(doc_id, score)] (sorted)
            vsrm: instance of `VectorSpaceRetrievalModel`
            similarity_fn: function instance that takes in two vectors 
                            and returns a similarity score e.g cosine_sim defined earlier
        """
        self.ret = initial_retrieval_fn
        self.vsrm = vsrm
        self.similarity_fn = similarity_fn
        self.vectorized_documents = vsrm.vectorize_documents()
        
        assert len(self.vectorized_documents) == len(doc_repr_2)
    
    def search(self, query, K=50):
        """
            First, retrieve the top K results using the retrieval function
            Then, re-rank the results using the VSRM instance
        """
        # YOUR CODE HERE
        raise NotImplementedError()

In [101]:
##### Function check
bm25_search_2 = partial(bm25_search, index_set=2)
lsi_rerank = DenseRerankingModel(bm25_search_2, lsi, cosine_sim)
lda_rerank = DenseRerankingModel(bm25_search_2, lda, jenson_shannon_sim)
w2v_rerank = DenseRerankingModel(bm25_search_2, w2v, cosine_sim)
w2v_pretrained_rerank = DenseRerankingModel(bm25_search_2, w2v_pretrained, cosine_sim)
d2v_rerank = DenseRerankingModel(bm25_search_2, d2v, cosine_sim)

##### 

<ipython-input-90-58fe68e8332b>:26: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if token in self.model:
<ipython-input-90-58fe68e8332b>:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  doc_dict[doc_id] += self.model[token]


\#### Please do not change this. This cell is used for grading.

---
Now, let us time the new search functions:

In [102]:
query = "algebraic functions"
print("BM25: ")
%timeit bm25_search(query, 2)
print("LSI: ")
%timeit lsi_rerank.search(query)
print("LDA: ")
%timeit lda_rerank.search(query)
print("W2V: ")
%timeit w2v_rerank.search(query)
print("W2V(Pretrained): ")
%timeit w2v_pretrained_rerank.search(query)
print("D2V:")
%timeit d2v_rerank.search(query)

BM25: 
9.15 ms ± 670 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
LSI: 


NotImplementedError: 

---
As you can see, it is much faster (but BM25 is still orders of magnitude faster).

---
## Section 10: Evaluation & Analysis (30 points) <a class="anchor" id="reranking_eval"></a>

[Back to Part 2](#part2)

[Previously](#evaluation) we have implemented some evaluation metrics and used them for measuring the ranking performance of term-based IR algorithms. In this section, we will do the same for semantic methods, both with and without re-ranking.

### Section 10.1: Plot (10 points)

First, gather the results. The results should consider the index set, the different search functions and different metrics. Plot the results in bar charts, per metric, with clear labels.

Then, gather only the re-ranking models, and plot and compare them with the results obtained in part 1 (only index set 2).

In [ ]:
list_of_sem_search_fns = [
    ("lda", drm_lda.search),
    ("lsi", drm_lsi.search),
    ("w2v", drm_w2v.search),
    ("w2v_pretrained", drm_w2v_pretrained.search),
    ("d2v", drm_d2v.search),
    ("lsi_rr", lsi_rerank.search),
    ("lda_rr", lda_rerank.search),
    ("w2v_rr", w2v_rerank.search),
    ("w2v_pretrained_rr", w2v_pretrained_rerank.search),
    ("d2v_rr", d2v_rerank.search),
    
]

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### Section 10.2: Summary (20 points)

Your summary should compare methods from Part 1 and Part 2 (only for index set 2). State what you expected to see in the results, followed by either supporting evidence *or* justify why the results did not support your expectations. Consider the availability of data, scalability, domain/type of data, etc.

YOUR ANSWER HERE